In [1]:
import numpy as np
import cv2
import glob

In [2]:
from queue import Queue
from threading import Thread
import sys
import time

class ImageReader:
    def __init__(self, image_file_list, queue_size=100):
        # initialize the file video stream along with the boolean
        # used to indicate if the thread should be stopped or not
        self.image_files = image_file_list
        self.total_frames = len(image_file_list)
        self.stopped = False
        # initialize the queue used to store frames read from
        # the video file
        self.Q = Queue(maxsize=queue_size)

    def start(self):
        # start a thread to read frames from the file video stream
        t = Thread(target=self.update, args=())
        t.daemon = True
        t.start()
        time.sleep(1)
        return self

    def update(self):
        # keep looping infinitely
        frame_num = 0
        while True:
            # if the thread indicator variable is set, stop the
            # thread
            if self.stopped:
                return

            # otherwise, ensure the queue has room in it
            if not self.Q.full():
                # read the next frame from the file
                image_np = cv2.imread(self.image_files[frame_num])
                # add the frame to the queue
                self.Q.put(image_np)
                
                frame_num += 1
                
                if frame_num >= self.total_frames:
                    self.stop()
                    return
                
                

    def read(self):
        # return next frame in the queue
        return self.Q.get()

    def read_batch(self, n_frames, asarray=False):
        frames = []
        for idx in range(n_frames):
            frames.append(self.read())
        return frames


    def more(self):
        # return True if there are still frames in the queue
        return self.Q.qsize() > 0

    def stop(self):
        # indicate that the thread should be stopped
        print('stopping')
        self.stopped = True
        
    def close(self):
        self.stop()

In [2]:
image_folder = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/long-buffalo-pictures'   
image_files = glob.glob(image_folder + '/*.jpg')
# image_files = glob.glob(image_folder + '*')
image_files.sort()
#positions_folder = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/test-frame/'
positions_folder = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/long-buffalo-position/'

print(len(image_files), 'images found')


39644 images found


In [3]:
# image1 = cv2.imread(image_files[18224])
# image2 = cv2.imread(image_files[18244])
image1 = cv2.imread(image_files[0])
image2 = cv2.imread(image_files[20])
frame_width, frame_height, _ = image1.shape
print('frame width: ', frame_width)
print('frame height: ', frame_height)


frame width:  2160
frame height:  4096


In [4]:
im1_gray = np.float32(cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY))
im2_gray = np.float32(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY))

In [32]:
h,w = im1_gray.shape

import time

t = time.time()
ret_list = []
image1 = cv2.imread(image_files[10000])
image2 = cv2.imread(image_files[10001])
for i in range(2):
#     image1 = cv2.imread(image_files[i])
#     image2 = cv2.imread(image_files[1+i])
    im1_gray = np.float32(cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY))
    im2_gray = np.float32(cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY))
#     im1_gray =  np.float32(image1[:,:,0])
#     im2_gray =  np.float32(image2[:,:,0])
    
    ret, _= cv2.phaseCorrelate(im1_gray, im2_gray)
    print(np.sum(np.array(ret)))
    ret_list.append(ret)
#     M = np.float32([[1, 0, -ret[0]], [0, 1, -ret[1]]])

#     trans_im = cv2.warpAffine(im2_gray, M, (w,h) )
print(time.time() - t)

0.0818964451846
0.0818964451846
0.6082677841186523


In [94]:
print(ret)   #247.4174063205719

(0.01673893698239226, 0.16426711320605136)


In [52]:
cv2.imshow('frame2',im1_gray/255)
k = cv2.waitKey(5000) & 0xff
cv2.imshow('frame2',trans_im/255)
k = cv2.waitKey(10000) & 0xff

cv2.destroyAllWindows()

In [96]:
sum(i[0] for i in ret_list)

0.1349457064748094

In [90]:
sum(ret)

[(-0.09400151557792924, 0.06933238074566361),
 (0.043807069905369644, 0.02104972241613723),
 (-0.023214323574848095, 0.05102297802841349),
 (0.20236410114398495, 0.033223736140143956),
 (0.06914153884167717, 0.08847964639471684),
 (-0.05933737644454595, 0.06370580572388462),
 (0.14329789817020355, 0.07596730587965794),
 (-0.09675107733983168, 0.0771684149478915),
 (0.019025654397410108, -0.10502045675116278),
 (-0.015526621411027008, -0.038347711747974245),
 (-0.16242073049807004, -0.07732315428256697),
 (0.20523085945319508, 0.00550672224653681),
 (-0.03315071963561422, 0.05181215880020318),
 (0.025767238468688447, 0.04947707694236669),
 (-0.004989530523289432, 0.0914759161105394),
 (0.03365702254222924, 0.007924954422605879),
 (-0.052745512195997435, 0.01098781637938373),
 (0.25679647623701385, 0.0466171080715867),
 (-0.06236290448623549, 0.02209005870781766),
 (0.007805670187281066, 0.07898430126465428)]